In [60]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

In [61]:
# get data in desirved format
dow_files_addr = "./Data/Dow_30_1_min/"
etf_files_addr = "./Data/50_ETFs_1min/"

dow_files_names = [f for f in listdir(dow_files_addr)]
etf_files_names = [f for f in listdir(etf_files_addr)]

dow_names = [f.split(".")[0] for f in dow_files_names]
etf_names = [f.split(".")[0] for f in etf_files_names]

dow_dfs = []
etf_dfs = []
headers = ["date", "time", "open_price", "close_price", "highest_price", "lowest_price", "volumn"]
dtypes={"date": "str", 
    "time":"str",
    "open_price": "float", 
    "highest_price": "float",
    "lowest_price": "float", 
    "close_price": "float",
    "volumn": "int"}

print("Start read dow file...")
for dow_name in dow_files_names:
    df = pd.read_csv(dow_files_addr + dow_name, sep=",",names=headers,dtype=dtypes, header=None)
    df["datetime"] = df["date"] + " " + df["time"]
    df["datetime"] = pd.to_datetime(df["datetime"], format="%m/%d/%Y %H:%M")
    df["minute"] = df["time"].str.split(":", expand=True)[1].astype(int)
    df = df.sort_values(by=["datetime"])
    dow_dfs.append(df)
print("Finish read dow file.")
print("---------------------")
print("Start read etf file...")
for etf_name in etf_files_names:
    df = pd.read_csv(etf_files_addr + etf_name, sep=",",names=headers,dtype=dtypes, header=None)
    df["datetime"] = df["date"] + " " + df["time"]
    df["datetime"] = pd.to_datetime(df["datetime"], format="%m/%d/%Y %H:%M")
    df["minute"] = df["time"].str.split(":", expand=True)[1].astype(int)
    df = df.sort_values(by=["datetime"])
    etf_dfs.append(df)
print("Finish read etf file.")

Start read dow file...
Finish read dow file.
---------------------
Start read etf file...
Finish read etf file.


In [62]:
# get start and end for both dow and etf that cove evey df

dow_time_slice = [min(dow_dfs[0]["datetime"]), max(dow_dfs[0]["datetime"])]
etf_time_slice = [min(etf_dfs[0]["datetime"]), max(etf_dfs[0]["datetime"])]

print("Start check dow date...")
i = 0
for df in dow_dfs:
    start = min(df["datetime"])
    if start > dow_time_slice[0]:
        print("start", dow_files_names[i])
        print(start)
        dow_time_slice[0] = start
    end = max(df["datetime"])
    if end < dow_time_slice[1]:
        print("end", dow_files_names[i])
        print(end)
        dow_time_slice[1] = end
    i += 1
print("Finish check dow date...")
print("---------------------")
print("Start check etf date...")
i = 0
for df in etf_dfs:
    start = min(df["datetime"])
    if start > etf_time_slice[0]:
        print("start", dow_files_names[i])
        print(start)
        etf_time_slice[0] = start
    end = max(df["datetime"])
    if end < etf_time_slice[1]:
        print("end", dow_files_names[i])
        print(end)
        etf_time_slice[1] = end
    i += 1
print("Finish check etf date...")

Start check dow date...
start GM.txt
2010-11-18 09:36:00
end GM.txt
2020-06-05 19:44:00
Finish check dow date...
---------------------
Start check etf date...
start AAPL.txt
2003-04-14 09:32:00
end AAPL.txt
2020-06-05 19:41:00
end AIG.txt
2020-06-05 17:38:00
start AXP.txt
2008-11-19 09:30:00
start T.txt
2009-06-25 09:40:00
Finish check etf date...


In [63]:
# limit all of the df to the same time frames
for i in range(len(dow_dfs)):
    dow_dfs[i] = dow_dfs[i].loc[(dow_dfs[i]["datetime"] >= dow_time_slice[0]) & (dow_dfs[i]["datetime"] <= dow_time_slice[1])]
for i in range(len(etf_dfs)):
    etf_dfs[i] = etf_dfs[i].loc[(etf_dfs[i]["datetime"] >= etf_time_slice[0]) & (etf_dfs[i]["datetime"] <= etf_time_slice[1])]

In [9]:
dow_time_slice

[Timestamp('2010-11-18 09:36:00'), Timestamp('2020-06-05 19:44:00')]

In [10]:
etf_time_slice

[Timestamp('2009-06-25 09:40:00'), Timestamp('2020-06-05 17:38:00')]

In [64]:
[Timestamp('2010-11-18 09:36:00'), Timestamp('2020-06-05 19:44:00')]
[Timestamp('2009-06-25 09:40:00'), Timestamp('2020-06-05 17:38:00')]

NameError: name 'Timestamp' is not defined

In [56]:
df = dow_dfs[0]

In [57]:
df

,date,time,open_price,close_price,highest_price,lowest_price,volumn,datetime
1277889,11/18/2010,09:36,36.59,36.68,36.58,36.63,45685,2010-11-18 09:36:00
1277890,11/18/2010,09:37,36.63,36.64,36.57,36.64,40420,2010-11-18 09:37:00
1277891,11/18/2010,09:38,36.65,36.65,36.57,36.61,26309,2010-11-18 09:38:00
1277892,11/18/2010,09:39,36.61,36.62,36.55,36.55,46309,2010-11-18 09:39:00
1277893,11/18/2010,09:40,36.55,36.56,36.52,36.54,70155,2010-11-18 09:40:00
...,...,...,...,...,...,...,...,...
2247188,06/05/2020,18:51,109.36,109.36,109.36,109.36,3537,2020-06-05 18:51:00
2247189,06/05/2020,18:54,109.95,109.95,109.95,109.95,500,2020-06-05 18:54:00
2247190,06/05/2020,19:11,109.77,109.77,109.77,109.77,900,2020-06-05 19:11:00
2247191,06/05/2020,19:15,109.77,109.77,109.77,109.77,423,2020-06-05 19:15:00


In [58]:
df["time"] = df["time"].str.split(":", expand=True)[1].astype(int)

In [59]:
df["time"]

1277889    36
1277890    37
1277891    38
1277892    39
1277893    40
           ..
2247188    51
2247189    54
2247190    11
2247191    15
2247192    43
Name: time, Length: 969304, dtype: int64

In [41]:
tmp["minute"] = tmp["time"].str.contains(":")

NameError: name 'tmp' is not defined

In [48]:
1 in tmp["minute"]

True

In [50]:
.loc[df['column_name'] == some_value]

,date,time,open_price,close_price,highest_price,lowest_price,volumn,datetime,minute
1470600,06/05/2020,19:59,12.00,12.00,12.00,12.00,1800,2020-06-05 19:59:00,True
1470599,06/05/2020,19:56,12.00,12.00,12.00,12.00,400,2020-06-05 19:56:00,True
1470598,06/05/2020,19:55,12.04,12.04,12.04,12.04,1791,2020-06-05 19:55:00,True
1470597,06/05/2020,19:52,12.05,12.05,12.05,12.05,416,2020-06-05 19:52:00,True
1470596,06/05/2020,19:44,12.05,12.05,12.05,12.05,384,2020-06-05 19:44:00,True


In [52]:
df["minute"] = df["time"].str.split(":")

In [56]:
df["minute"][0][0]

'12'